In [ ]:
import pandas as pd
import os
import ast
import pydeck as pdk
import numpy as np

df = pd.read_csv('../data/raw/bangkok_traffy.csv')


In [ ]:
df.info()
df

In [ ]:


df = df[df['coords'].notna()]

def parse_coords(x):
    try:
        lat, lon = ast.literal_eval(x)
        return lat, lon
    except:
        return (np.nan, np.nan)

df[['lat', 'lon']] = df['coords'].apply(parse_coords).tolist()
df = df.dropna(subset=['lat', 'lon'])
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp'])
df = df.dropna(subset=['district', 'subdistrict'])
df = df.reset_index(drop=True)



In [ ]:
df['type'].value_counts()


In [ ]:
road_df = df[
    df['type'].str.contains('ถนน', na=False) &
    df['comment'].str.contains('ถนน', na=False)
].copy()
road_df[['latitude', 'longitude']] = road_df['coords'].apply(lambda x: pd.Series(ast.literal_eval(x)))

target_n = 100
road_df_small = road_df.dropna(subset=['latitude','longitude']).sample(
    n=min(target_n, len(road_df)),
    random_state=42
)



In [ ]:
heatmap_layer = pdk.Layer(
    "HeatmapLayer",
    data=road_df_small,
    get_position='[longitude, latitude]',
    radiusPixels=50,
    intensity=1,
    threshold=0.05,
    opacity=0.5
)

view_state = pdk.ViewState(
    latitude=road_df_small['latitude'].mean(),
    longitude=road_df_small['longitude'].mean(),
    zoom=11,
    pitch=45
)

pdk.Deck(
    layers=[heatmap_layer],
    initial_view_state=view_state,
    map_style='mapbox://styles/mapbox/light-v9'
)